In [1]:
import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

hac_mo = '10'

## Load HAC file

In [2]:
hac_df = pd.read_pickle(mydir + "data/CareCompare_data/CombinedFiles_HACRP/Facility.pkl")
hac_df = hac_df[(hac_df['file_year'].isin(['2017'])) & (hac_df['file_month'] == hac_mo)]
print('HACRP years:', sorted(hac_df['Fiscal Year'].unique()))

features = ['CAUTI Score', 'CLABSI Score', 'CDI Score', 'MRSA Score', 'SSI Score',
            'Total HAC Score', 'Domain 1 Score', 'AHRQ PSI-90 Score', 'Domain 2 Score',
           'CAUTI W Z Score', 'CLABSI W Z Score', 'MRSA W Z Score', 'CDI W Z Score', 'SSI W Z Score',
           'PSI-90 W Z Score']

for f in features:
    hac_df[f] = hac_df[f].astype(str)
    hac_df[f] = pd.to_numeric(hac_df[f], errors='coerce')

hac_df.head()

HACRP years: [2017]


,AHRQ PSI-90 Footnote,AHRQ PSI-90 Score,CAUTI Footnote,CAUTI Score,CAUTI W Z Score,CDI Footnote,CDI Score,CDI W Z Score,CLABSI Footnote,CLABSI Score,CLABSI W Z Score,Domain 1 End Date,Domain 1 Footnote,Domain 1 Score,Domain 1 Score Footnote,Domain 1 Start Date,Domain 2 End Date,Domain 2 Footnote,Domain 2 Score,Domain 2 Score Footnote,Domain 2 Start Date,Facility ID,Facility Name,Fiscal Year,Footnotes,HAI Measures End Date,HAI Measures Start Date,MRSA Footnote,MRSA Score,MRSA W Z Score,PSI-90 End Date,PSI-90 Footnote,PSI-90 Start Date,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI Score,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year
0,NaN,1.0,NaN,10.0,NaN,NaN,7.0,NaN,NaN,10.0,NaN,6302015,NaN,1.0,NaN,7012013,12312015,NaN,8.0,NaN,1012014,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2017,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,7.0,NaN,AL,NaN,6.95,10,2017
1,NaN,5.0,NaN,5.0,NaN,NaN,2.0,NaN,NaN,10.0,NaN,6302015,NaN,5.0,NaN,7012013,12312015,NaN,5.4,NaN,1012014,010005,MARSHALL MEDICAL CENTERS,2017,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,2.0,NaN,AL,NaN,5.34,10,2017
2,NaN,7.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,5.0,NaN,6302015,NaN,7.0,NaN,7012013,12312015,NaN,6.0,NaN,1012014,010006,ELIZA COFFEE MEMORIAL HOSPITAL,2017,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,8.0,NaN,AL,NaN,6.15,10,2017
3,NaN,3.0,NaN,1.0,NaN,NaN,1.0,NaN,5.0,NaN,NaN,6302015,NaN,3.0,NaN,7012013,12312015,NaN,1.0,NaN,1012014,010007,MIZELL MEMORIAL HOSPITAL,2017,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,5.0,NaN,NaN,AL,NaN,1.30,10,2017
4,NaN,6.0,5.0,NaN,NaN,NaN,6.0,NaN,5.0,NaN,NaN,6302015,NaN,6.0,NaN,7012013,12312015,NaN,6.0,NaN,1012014,010008,CRENSHAW COMMUNITY HOSPITAL,2017,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,5.0,NaN,NaN,AL,NaN,6.00,10,2017


## Format dates within the HAC file 

In [3]:
## Fix Nov 2017 date format and convert to default datetime
hac_df = hac_df[(hac_df['file_year'] == '2017') & (hac_df['file_month'] == hac_mo)]
hac_df['Domain 2 Start Date'] = '0' + hac_df['Domain 2 Start Date'].astype(str)
hac_df['Domain 2 Start Date'] = pd.to_datetime(hac_df['Domain 2 Start Date'], format='%m%d%Y')
hac_df.style.format({'Domain 2 Start Date': lambda t: t.strftime("%Y-%m-%d")})
hac_df['Domain 2 End Date'] = pd.to_datetime(hac_df['Domain 2 End Date'], format='%m%d%Y')
hac_df.style.format({'Domain 2 End Date': lambda t: t.strftime("%Y-%m-%d")})
hac_df.rename(columns={'Domain 2 Start Date': 'Start Date', 'Domain 2 End Date': 'End Date'}, inplace=True)
hac_df.sort_values(by='Start Date', inplace=True)


## Load and merge HAI files

In [4]:
##############################   CAUTI   ################################################

cauti_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CAUTI_Data.pkl")
cauti_df = cauti_df.filter(items=['Facility ID', 'CAUTI Urinary Catheter Days (ICUs only)', 
                                  'CAUTI Observed Cases (ICUs only)', 'CAUTI Predicted Cases (ICUs only)', 
                                  'Start Date', 'End Date'], axis=1)

cauti_df.rename(columns={'CAUTI Urinary Catheter Days (ICUs only)': 'CAUTI Urinary Catheter Days', 
                         'CAUTI Observed Cases (ICUs only)': 'CAUTI Observed Cases', 
                         'CAUTI Predicted Cases (ICUs only)': 'CAUTI Predicted Cases', 
                        }, inplace=True)

features = ['CAUTI Urinary Catheter Days', 'CAUTI Observed Cases', 'CAUTI Predicted Cases']
for f in features:
    cauti_df[f] = cauti_df[f].astype(str)
    cauti_df[f] = pd.to_numeric(cauti_df[f], errors='coerce')

cauti_df['Start Date'] = pd.to_datetime(cauti_df['Start Date'])
cauti_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df['End Date'] = pd.to_datetime(cauti_df['End Date'])
cauti_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df = cauti_df[(cauti_df['Start Date'].isin(['2014-01-01', '2015-01-01'])) | (cauti_df['End Date'].isin(['2014-01-01', '2015-01-01']))]

cauti_df.head()


,Facility ID,CAUTI Urinary Catheter Days,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date
1,260179,NaN,NaN,NaN,2015-01-01,2015-06-30
8,370056,4206.0,9.0,5.646,2014-01-01,2014-12-31
14,361328,NaN,NaN,NaN,2015-01-01,2015-09-30
21,450845,NaN,NaN,NaN,2015-01-01,2015-09-30
27,061302,NaN,NaN,NaN,2015-01-01,2015-09-30


In [5]:
##############################   CLABSI   ###############################################

clabsi_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CLABSI_Data.pkl")
clabsi_df = clabsi_df.filter(items=['Facility ID', 'CLABSI Device Days (ICUs only)', 
                                    'CLABSI Observed Cases (ICUs only)', 'CLABSI Predicted Cases (ICUs only)', 
                                    'Start Date', 'End Date'], axis=1)

clabsi_df.rename(columns={'CLABSI Device Days (ICUs only)': 'CLABSI Device Days', 
                         'CLABSI Observed Cases (ICUs only)': 'CLABSI Observed Cases', 
                         'CLABSI Predicted Cases (ICUs only)': 'CLABSI Predicted Cases', 
                        }, inplace=True)


features = ['CLABSI Device Days', 'CLABSI Observed Cases', 'CLABSI Predicted Cases']
for f in features:
    clabsi_df[f] = clabsi_df[f].astype(str)
    clabsi_df[f] = pd.to_numeric(clabsi_df[f], errors='coerce')

clabsi_df['Start Date'] = pd.to_datetime(clabsi_df['Start Date'])
clabsi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df['End Date'] = pd.to_datetime(clabsi_df['End Date'])
clabsi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df = clabsi_df[(clabsi_df['Start Date'].isin(['2014-01-01', '2015-01-01'])) | (clabsi_df['End Date'].isin(['2014-01-01', '2015-01-01']))]

clabsi_df.head()

,Facility ID,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,Start Date,End Date
10,370047,NaN,NaN,NaN,2015-01-01,2015-06-30
13,330396,2555.0,7.0,5.957,2014-01-01,2014-12-31
26,360141,NaN,NaN,NaN,2015-01-01,2015-09-30
28,450716,NaN,NaN,NaN,2015-01-01,2015-09-30
31,180009,2849.0,2.0,4.232,2015-01-01,2015-12-31


In [6]:
##############################   MRSA   ###############################################

mrsa_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/MRSA_Data.pkl")
mrsa_df = mrsa_df.filter(items=['Facility ID', 'MRSA patient days', 
                                'MRSA Observed Cases', 'MRSA Predicted Cases', 
                                'Start Date', 'End Date'], axis=1)

features = ['MRSA patient days', 'MRSA Observed Cases', 'MRSA Predicted Cases']
for f in features:
    mrsa_df[f] = mrsa_df[f].astype(str)
    mrsa_df[f] = pd.to_numeric(mrsa_df[f], errors='coerce')

mrsa_df['Start Date'] = pd.to_datetime(mrsa_df['Start Date'])
mrsa_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
mrsa_df['End Date'] = pd.to_datetime(mrsa_df['End Date'])
mrsa_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
mrsa_df = mrsa_df[(mrsa_df['Start Date'].isin(['2014-01-01', '2015-01-01'])) | (mrsa_df['End Date'].isin(['2014-01-01', '2015-01-01']))]


##############################   CDI   ###############################################

cdi_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CDI_Data.pkl")
cdi_df = cdi_df.filter(items=['Facility ID', 'CDIFF patient days', 
                              'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
                              'Start Date', 'End Date'], axis=1)

features = ['CDIFF patient days', 'CDIFF Observed Cases', 'CDIFF Predicted Cases']
for f in features:
    cdi_df[f] = cdi_df[f].astype(str)
    cdi_df[f] = pd.to_numeric(cdi_df[f], errors='coerce')

cdi_df['Start Date'] = pd.to_datetime(cdi_df['Start Date'])
cdi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cdi_df['End Date'] = pd.to_datetime(cdi_df['End Date'])
cdi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
cdi_df = cdi_df[(cdi_df['Start Date'].isin(['2014-01-01', '2015-01-01'])) | (cdi_df['End Date'].isin(['2014-12-31', '2015-12-31']))]


###################  Merge CAUTI, CLABSI, MRSA, and CDI  ####################################

hai_df = cauti_df.merge(clabsi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(mrsa_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(cdi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')


####################  Drop duplicate rows resulting from merger #############################

hai_df.drop_duplicates(inplace=True)

########################  Conversions to numeric  #############################################

features = ['CAUTI Urinary Catheter Days', 'CLABSI Device Days',
            'MRSA patient days', 'CDIFF patient days',
            'CLABSI Observed Cases', 'CLABSI Predicted Cases', 
            'MRSA Observed Cases', 'MRSA Predicted Cases', 
            'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
            'CAUTI Observed Cases', 'CAUTI Predicted Cases', 
           ]

for f in features:
    hai_df[f] = hai_df[f].astype(str)
    hai_df[f] = hai_df[f].str.replace('*', '')
    hai_df[f] = hai_df[f].str.replace(' ', '')
    hai_df[f] = pd.to_numeric(hai_df[f], errors='coerce')
    
hai_df['Total device days'] = hai_df['CLABSI Device Days'] + hai_df['CAUTI Urinary Catheter Days']


########################  Reorder columns  #############################################

col_to_move = hai_df.pop('CAUTI Urinary Catheter Days')
hai_df.insert(hai_df.shape[1] - 2, 'CAUTI Urinary Catheter Days', col_to_move)

hai_df.head()

,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days
0,260179,NaN,NaN,2015-01-01,2015-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,370056,9.0,5.646,2014-01-01,2014-12-31,3058.0,4.0,4.501,41695.0,3.0,2.567,37251.0,19.0,4206.0,26.396,7264.0
2,361328,NaN,NaN,2015-01-01,2015-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,450845,NaN,NaN,2015-01-01,2015-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,061302,NaN,NaN,2015-01-01,2015-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filter HAI data on start dates and end dates that match those in the HAC file. Then, drop hospitals from HAI data that are not contained in HAC data.

In [7]:
#########  Filter on start dates and end dates that match those in the HAC file  #######

hai_df = hai_df[(hai_df['Start Date'].isin(['2014-01-01', '2015-01-01'])) & (hai_df['End Date'].isin(['2014-12-31', '2015-12-31']))]
print(hai_df['Start Date'].unique())
print(hai_df['End Date'].unique())

tdf = hac_df[~hac_df['Facility ID'].isin(hai_df['Facility ID'].unique())]
print(2017, ' : ', len(tdf['Facility ID'].unique()), 'hospitals in HAC dataset but not in HAI dataset\n')
    
hai_df = hai_df[hai_df['Facility ID'].isin(hac_df['Facility ID'].unique())]


['2014-01-01T00:00:00.000000000' '2015-01-01T00:00:00.000000000']
['2014-12-31T00:00:00.000000000' '2015-12-31T00:00:00.000000000']
2017  :  1 hospitals in HAC dataset but not in HAI dataset



## Correct HAI file for non-duplicate rows having duplicate dates

**Problem:** Some rows for the same provider have duplicate measurement dates but different values for observed cases, predicted cases, etc. This results from each year having multiple (quarterly) files, the data within which can vary among files. Additionally, the most recent file for each year is not always the right file to use. 

**Need:** Since only one row can be used, we need to figure out which row should be used.

**Solution:** Select the row with the greatest totals for predicted cases for each HAI.

In [8]:
##############  Label rows that have duplicate dates (per provider) ####################
##############  For each provider with rows having duplicate dates,  ###################
###########  keep the last row (will have greatest number of total device days)  #######

hai_df['duplicated dates'] = hai_df.duplicated(subset=['Facility ID', 'Start Date', 'End Date'], keep=False)

hai_df.sort_values(by=['Facility ID', 'Start Date', 'End Date',
                       'MRSA Predicted Cases', 'MRSA Observed Cases',
                       'CDIFF Predicted Cases', 'CDIFF Observed Cases',
                       'CLABSI Predicted Cases', 'CLABSI Observed Cases',
                       'CAUTI Predicted Cases',  'CAUTI Observed Cases',
                       ], inplace=True, ascending=False)

hai_df.tail(21)

,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days,duplicated dates
53360,010005,NaN,NaN,2015-01-01,2015-12-31,1028.0,1.0,1.543,36123.0,2.0,1.094,36262.0,7.0,NaN,10.957,NaN,True
53372,010005,3.0,4.297,2015-01-01,2015-12-31,NaN,NaN,NaN,36123.0,2.0,1.094,36262.0,7.0,3305.0,10.957,NaN,True
53364,010005,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,36123.0,2.0,1.094,36262.0,7.0,NaN,10.957,NaN,True
93823,010005,2.0,3.831,2014-01-01,2014-12-31,870.0,4.0,1.306,38084.0,2.0,1.610,36227.0,5.0,2946.0,15.840,3816.0,False
27702,010001,18.0,7.803,2015-01-01,2015-12-31,2191.0,8.0,3.288,83773.0,3.0,7.111,81130.0,37.0,6502.0,56.348,8693.0,True
27710,010001,NaN,NaN,2015-01-01,2015-12-31,2191.0,8.0,3.288,83773.0,3.0,7.111,81130.0,37.0,NaN,56.348,NaN,True
27698,010001,18.0,7.803,2015-01-01,2015-12-31,NaN,NaN,NaN,83773.0,3.0,7.111,81130.0,37.0,6502.0,56.348,NaN,True
27706,010001,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,83773.0,3.0,7.111,81130.0,37.0,NaN,56.348,NaN,True
27703,010001,18.0,7.803,2015-01-01,2015-12-31,2191.0,8.0,3.288,83773.0,3.0,7.111,81130.0,37.0,6502.0,47.018,8693.0,True
27711,010001,NaN,NaN,2015-01-01,2015-12-31,2191.0,8.0,3.288,83773.0,3.0,7.111,81130.0,37.0,NaN,47.018,NaN,True


In [9]:
hai_df.drop_duplicates(subset=['Facility ID', 'Start Date', 'End Date'], inplace=True, keep='first')
hai_df.head(7)

,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days,duplicated dates
46009,670114,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
42536,670112,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
14201,670111,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
54296,670109,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
42436,670108,0.0,0.055,2015-01-01,2015-12-31,34.0,0.0,0.051,948.0,0.0,0.034,875.0,0.0,42.0,0.428,76.0,True
28660,670107,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,44.0,0.0,0.002,44.0,0.0,NaN,0.019,NaN,True
77603,670106,NaN,NaN,2015-01-01,2015-12-31,NaN,NaN,NaN,1175.0,0.0,0.042,1154.0,1.0,NaN,0.676,NaN,True


## Aggregate annual HAI data into biennial data

Purpose: Match the biennial measurement periods of HAC data


In [10]:
start_dates = hac_df['Start Date'].tolist()
end_dates = hac_df['End Date'].tolist()
prvdrs = hac_df['Facility ID'].tolist()

total_device_days = []

cauti_days = []
clabsi_days = []
mrsa_days = []
cdi_days = []

cauti_pred = []
clabsi_pred = []
mrsa_pred = []
cdi_pred = []

cauti_obs = []
clabsi_obs = []
mrsa_obs = []
cdi_obs = []

ct1 = 0
limited_prvdrs = []

for i, start in enumerate(start_dates):
    end = end_dates[i]
    prvdr = prvdrs[i]
    
    tdf = hai_df[hai_df['Facility ID'] == prvdr]
    #tdf = tdf[(tdf['Start Date'] == start) | (tdf['End Date'] == end)]
    
    if tdf.shape[0] == 1:
        ct1 += 1
        limited_prvdrs.append(prvdr)
    
    '''
    if tdf.shape[0] > 2:
        print('tdf.shapej[0] > 2:', prvdr)
        for date_ in ['Start Date', 'End Date']:
            tdf['duplicated dates'] = tdf.duplicated(subset=[date_], keep=False)
            tdf.sort_values(by=['MRSA Predicted Cases', 'CAUTI Predicted Cases',  
                                'MRSA Observed Cases', 'CAUTI Observed Cases', 
                                'CDIFF Predicted Cases', 'CLABSI Predicted Cases',
                                'CDIFF Observed Cases', 'CLABSI Observed Cases',
                                ], inplace=True, ascending=False)

            tdf.drop_duplicates(subset=[date_], inplace=True, keep='first')

        if tdf.shape[0] > 2:
            print('Error:')
            print("tdf.shape[0] > 2:", tdf.shape[0])
            print(start)
            print(end)
            print(tdf.head())
            break
    '''
    
    total_device_days.append(np.nansum(tdf['Total device days']))
    cauti_days.append(np.nansum(tdf['CAUTI Urinary Catheter Days']))
    clabsi_days.append(np.nansum(tdf['CLABSI Device Days']))
    mrsa_days.append(np.nansum(tdf['MRSA patient days']))
    cdi_days.append(np.nansum(tdf['CDIFF patient days']))
        
    cauti_pred.append(np.nansum(tdf['CAUTI Predicted Cases']))
    clabsi_pred.append(np.nansum(tdf['CLABSI Predicted Cases']))
    mrsa_pred.append(np.nansum(tdf['MRSA Predicted Cases']))
    cdi_pred.append(np.nansum(tdf['CDIFF Predicted Cases']))

    cauti_obs.append(np.nansum(tdf['CAUTI Observed Cases']))
    clabsi_obs.append(np.nansum(tdf['CLABSI Observed Cases']))
    mrsa_obs.append(np.nansum(tdf['MRSA Observed Cases']))
    cdi_obs.append(np.nansum(tdf['CDIFF Observed Cases']))
    
print(ct1)

52


## Add HAI data to the HAC dataframe and save

In [11]:
hac_df['Total device days'] = total_device_days
hac_df['CAUTI Urinary Catheter Days'] = cauti_days
hac_df['CLABSI Device Days'] = clabsi_days
hac_df['MRSA patient days'] = mrsa_days
hac_df['CDI patient days'] = cdi_days

hac_df['CAUTI Observed Cases'] = cauti_obs
hac_df['CLABSI Observed Cases'] = clabsi_obs
hac_df['MRSA Observed Cases'] = mrsa_obs
hac_df['CDI Observed Cases'] = cdi_obs

hac_df['CAUTI Predicted Cases'] = cauti_pred
hac_df['CLABSI Predicted Cases'] = clabsi_pred
hac_df['MRSA Predicted Cases'] = mrsa_pred
hac_df['CDI Predicted Cases'] = cdi_pred

hac_df['CAUTI derived SIR'] = hac_df['CAUTI Observed Cases'] / hac_df['CAUTI Predicted Cases']
hac_df['CLABSI derived SIR'] = hac_df['CLABSI Observed Cases'] / hac_df['CLABSI Predicted Cases']
hac_df['MRSA derived SIR'] = hac_df['MRSA Observed Cases'] / hac_df['MRSA Predicted Cases']
hac_df['CDI derived SIR'] = hac_df['CDI Observed Cases'] / hac_df['CDI Predicted Cases']


## Reproduce HAC scores and penalty assignments for 2017

In [12]:
df_2017 = hac_df[hac_df['file_year'] == '2017']
df_2017.dropna(how='all', axis=1, inplace=True)

hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']
start_dates = df_2017['Start Date'].unique()

new_df = 0
    
for hai in hais:
    tdf2 = df_2017[~df_2017[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf2.shape[0] > 0:
            
        derived_sirs = tdf2[hai + ' derived SIR'].tolist()
        reported_scores = tdf2[hai + ' Score'].tolist()
        derived_scores = []

        for ii, sir in enumerate(derived_sirs):
            if np.isnan(sir) == True or np.isnan(reported_scores[ii]) == True:
                derived_scores.append(np.nan)
            elif hai == 'CLABSI':
                score = np.nan
                if sir == 0.0:
                    score = 1
                elif sir <= 0.125:
                    score = 2
                elif sir <= 0.24:
                    score = 3
                elif sir <= 0.325:
                    score = 4
                elif sir <= 0.4055:
                    score = 5
                elif sir <= 0.502:
                    score = 6
                elif sir <= 0.613:
                    score = 7
                elif sir <= 0.753:
                    score = 8
                elif sir <= 0.992:
                    score = 9
                elif sir > 0.992:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            elif hai == 'CAUTI':
                score = np.nan
                if sir == 0.0:
                    score = 1
                elif sir <= 0.222:
                    score = 2
                elif sir <= 0.407:
                    score = 3
                elif sir <= 0.558:
                    score = 4
                elif sir <= 0.697:
                    score = 5
                elif sir <= 0.845:
                    score = 6
                elif sir <= 1.035:
                    score = 7
                elif sir <= 1.261:
                    score = 8
                elif sir <= 1.596:
                    score = 9
                elif sir > 1.596:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
            
            elif hai == 'MRSA':
                score = np.nan
                if sir == 0.0:
                    score = 1
                elif sir <= 0.298:
                    score = 2
                elif sir <= 0.478:
                    score = 3
                elif sir <= 0.623:
                    score = 4
                elif sir <= 0.763:
                    score = 5
                elif sir <= 0.923:
                    score = 6
                elif sir <= 1.104:
                    score = 7
                elif sir <= 1.344:
                    score = 8
                elif sir <= 1.722:
                    score = 9
                elif sir > 1.722:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)
                
                
            elif hai == 'CDI':
                score = np.nan
                if sir < 0.168:
                    score = 1
                elif sir <= 0.425:
                    score = 2
                elif sir <= 0.582:
                    score = 3
                elif sir <= 0.705:
                    score = 4
                elif sir <= 0.807:
                    score = 5
                elif sir <= 0.915:
                    score = 6
                elif sir <= 1.031:
                    score = 7
                elif sir <= 1.173:
                    score = 8
                elif sir <= 1.382:
                    score = 9
                elif sir > 1.382:
                    score = 10
                else:
                    score = np.nan
                            
                derived_scores.append(score)


        tdf2[hai + ' derived score'] = derived_scores
        print('len:', len(derived_scores), len(reported_scores))

    # Assign maximum scores to hospitals with HAI footnote 18 
    tdf3 = df_2017[df_2017[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    if tdf3.shape[0] > 0:
        tdf3[hai + ' derived score'] = 10
        df_2017 = pd.concat([tdf2, tdf3], axis=0)
            
    else:
        df_2017 = tdf2.copy(deep=True)

features = ['CAUTI derived SIR', 'CAUTI Score', 'CAUTI derived score',
            'CLABSI derived SIR', 'CLABSI Score', 'CLABSI derived score',
            'SSI Score', 'AHRQ PSI-90 Score',
            'CDI derived SIR', 'CDI Score', 'CDI derived score',
            'MRSA derived SIR', 'MRSA Score', 'MRSA derived score',
           ]

for f in features:
    df_2017[f] = pd.to_numeric(df_2017[f], errors='coerce')

df_2017.head()

len: 3297 3297
len: 3297 3297
len: 3290 3290
len: 3290 3290


,AHRQ PSI-90 Footnote,AHRQ PSI-90 Score,CAUTI Footnote,CAUTI Score,CDI Footnote,CDI Score,CLABSI Footnote,CLABSI Score,Domain 1 End Date,Domain 1 Score,Domain 1 Score Footnote,Domain 1 Start Date,End Date,Domain 2 Score,Domain 2 Score Footnote,Start Date,Facility ID,Facility Name,Fiscal Year,MRSA Footnote,MRSA Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,MRSA patient days,CDI patient days,CAUTI Observed Cases,CLABSI Observed Cases,MRSA Observed Cases,CDI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,MRSA Predicted Cases,CDI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,MRSA derived SIR,CDI derived SIR,CAUTI derived score,CLABSI derived score,MRSA derived score,CDI derived score
0,NaN,1.0,NaN,10.0,NaN,7.0,NaN,10.0,6302015,1.0,NaN,7012013,2015-12-31,8.0,NaN,2014-01-01,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2017,NaN,6.0,Yes,NaN,NaN,7.0,AL,NaN,6.95,10,2017,18260.0,13811.0,4449.0,182743.0,176802.0,50.0,18.0,12.0,94.0,16.583,6.676,14.625,118.846,3.015136,2.696225,0.820513,0.790940,10.0,10.0,6.0,5.0
2203,NaN,8.0,NaN,4.0,NaN,6.0,NaN,1.0,6302015,8.0,NaN,7012013,2015-12-31,4.6,NaN,2014-01-01,360082,EUCLID HOSPITAL,2017,NaN,9.0,No,NaN,NaN,3.0,OH,NaN,5.11,10,2017,5197.0,3038.0,2159.0,76300.0,76300.0,2.0,0.0,6.0,48.0,3.951,3.240,3.882,56.796,0.506201,0.000000,1.545595,0.845130,4.0,1.0,9.0,6.0
2204,NaN,9.0,NaN,2.0,NaN,3.0,NaN,9.0,6302015,9.0,NaN,7012013,2015-12-31,4.6,NaN,2014-01-01,360084,AULTMAN HOSPITAL,2017,NaN,5.0,No,NaN,NaN,4.0,OH,NaN,5.26,10,2017,26750.0,15786.0,10964.0,224014.0,199995.0,4.0,17.0,9.0,61.0,33.395,21.023,14.177,130.309,0.119778,0.808638,0.634831,0.468118,2.0,9.0,5.0,3.0
2205,NaN,8.0,NaN,8.0,NaN,4.0,NaN,6.0,6302015,8.0,NaN,7012013,2015-12-31,6.0,NaN,2014-01-01,360085,OHIO STATE UNIVERSITY STATE HEALTH SYSTEM,2017,NaN,6.0,No,NaN,NaN,6.0,OH,NaN,6.30,10,2017,87951.0,46810.0,41141.0,549053.0,508360.0,136.0,38.0,55.0,327.0,122.621,92.514,61.561,466.866,1.109109,0.410749,0.893423,0.700415,8.0,6.0,6.0,4.0
2206,NaN,3.0,NaN,2.0,NaN,8.0,NaN,2.0,6302015,3.0,NaN,7012013,2015-12-31,4.8,NaN,2014-01-01,360086,SPRINGFIELD REGIONAL MEDICAL CENTER,2017,NaN,10.0,No,NaN,NaN,2.0,OH,NaN,4.53,10,2017,22754.0,12443.0,10311.0,129444.0,123040.0,2.0,1.0,22.0,83.0,24.886,19.866,6.316,80.769,0.080366,0.050337,3.483217,1.027622,2.0,2.0,10.0,7.0


In [13]:
holdout_df = df_2017[(df_2017['State'] == 'MD') | ~df_2017['Payment Reduction'].isin(['Yes', 'No']) | (df_2017['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_2017 = df_2017[(df_2017['State'] != 'MD') & (df_2017['Payment Reduction'].isin(['Yes', 'No'])) & (~df_2017['Total HAC Score'].isin([float("NaN"), np.nan]))]

hac_scores = []
ct1 = 0
ct2 = 0

for hosp in df_2017['Facility ID'].tolist():
    tdf = df_2017[df_2017['Facility ID'] == hosp]

    d1 = 0
    d2 = 0
        
    w_ls = []
    sum_ls = []

    # Use original scores for all HAIs for a specific type of testing, i.e., can penalties be reproduced when using data directly from the HAC files
    m_ls = ['CAUTI derived score', 'CLABSI derived score', 'MRSA derived score', 'CDI derived score', 'SSI Score']
    
    # Use original SSI scores but derived scores for CDI, CAUTI, CLABSI, and MRSA for actual results
    #m_ls = ['CAUTI Score', 'CLABSI Score', 'MRSA Score', 'CDI Score', 'SSI Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].tolist()
        if len(list(set(v))) > 1:
            print('len(list(set(v))) > 1')
            sys.exit()

        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d2 = np.nan
    else:
        d2 = s/w
        
    d1 = float(tdf['AHRQ PSI-90 Score'].iloc[0])
        
    d2_o = float(tdf['Domain 2 Score'].iloc[0])
    state = tdf['State'].iloc[0]
    
    if np.isnan(d1) == True: 
        # if no score for Domain 1, then total HAC score will be based entirely on Domain 2
        if np.isnan(d2) == True and np.isnan(d2_o) == True:
            hac_scores.append(d2)
        
        elif np.isnan(d2) == True and np.isnan(d2_o) == False:
            hac_scores.append(d2_o)
        
        elif np.isnan(d2) == False and np.isnan(d2_o) == False:
            hac_scores.append(d2)
        
        elif np.isnan(d2) == False and np.isnan(d2_o) == True:
            hac_scores.append(d2_o)
        
    elif np.isnan(d1) == False:
        # if there is a score for Domain 1 ...
        
        if state == 'MD':
            # states in MD should not have scores for domain 1
            print('Error:')
            print('Domain 1:', d1)
            print('State:', state)
            print('Hospitals in this state should not have domain 1 scores.\n')
            
        # Domain 2:
        
        # If the derived score is NaN and the original score is NaN ...
        if np.isnan(d2) == True and np.isnan(d2_o) == True:
            hac_scores.append(d1)
        
        # If the derived score is a float but the original score is NaN ...
        elif np.isnan(d2) == False and np.isnan(d2_o) == True:
            hac_scores.append(d1)

        # If the derived score is NaN but the original score is a float ...
        elif np.isnan(d2) == True and np.isnan(d2_o) == False:
            ct1 += 1
            hac_scores.append(0.15*d1 + 0.85*d2_o)
        
        # If the derived score is a float and the original score is a float ...
        elif np.isnan(d2) == False and np.isnan(d2_o) == False:
            if d2 != d2_o:
                ct2 += 1
                #print(d2, d2_o)
            hac_scores.append(0.15*d1 + 0.85*d2)
        
    else:
        print(d1, ',', d2, ',', d2_o)
        break

print('ct1:', ct1)
print('ct2:', ct2)



ct1: 1
ct2: 759


In [14]:
df_2017['Total HAC Score (derived)'] = hac_scores
print(df_2017.shape[0], 'hospitals in hac_df')

tdf = df_2017[~df_2017['Total HAC Score (derived)'].isin([np.nan, float('NaN')])]
p75 = np.percentile(tdf['Total HAC Score (derived)'], 75, method='linear')
p75 = 6.5700 # HACRP had modified its 75th percentile to this

pr = []
for hosp in df_2017['Facility ID'].tolist():
    tdf = df_2017[df_2017['Facility ID'] == hosp]

    p = tdf['Payment Reduction'].iloc[0]
    if p != 'Yes' and p != 'No' and np.isnan(p) == True:
        pr.append(np.nan)

    else:
        score = tdf['Total HAC Score (derived)'].iloc[0]

        if np.isnan(score) == True:
            pr.append('No')
        elif score <= p75:
            pr.append('No')
        elif score > p75:
            pr.append('Yes')
        else:
            print('This score is an error:', score)
            sys.exit()
            

df_2017['Payment Reduction (derived)'] = pr
            
o_list = df_2017['Payment Reduction'].tolist()
d_list = df_2017['Payment Reduction (derived)'].tolist()

same = 0
diff = 0
res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        res_ls.append(0)
            
df_2017['Payment Reduction Reproduced?'] = res_ls
    
print(same, "Penalty assignments were reproduced")
print(diff, "Penalty assignments were not reproduced")
print(str(np.round(100 * same/(same+diff),2)) + '% penalty assignments were reproduced\n')
    
df_2017['HAC delta'] = df_2017['Total HAC Score'] - df_2017['Total HAC Score (derived)']
df_2017['CDI delta'] = df_2017['CDI Score'] - df_2017['CDI derived score']
df_2017['MRSA delta'] = df_2017['MRSA Score'] - df_2017['MRSA derived score']
df_2017['CAUTI delta'] = df_2017['CAUTI Score'] - df_2017['CAUTI derived score']
df_2017['CLABSI delta'] = df_2017['CLABSI Score'] - df_2017['CLABSI derived score']


3202 hospitals in hac_df
3159 Penalty assignments were reproduced
43 Penalty assignments were not reproduced
98.66% penalty assignments were reproduced



In [15]:
display_df = df_2017[df_2017['Payment Reduction Reproduced?'] == 0]
items = ['Facility ID', 'file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
             'CAUTI Footnote', 'CAUTI Score', 'CAUTI derived score',
             'CLABSI Footnote', 'CLABSI Score', 'CLABSI derived score',
             'CDI Footnote', 'CDI Score', 'CDI derived score',
             'MRSA Footnote', 'MRSA Score', 'MRSA derived score',
             'SSI Footnote', 'SSI Score',
             'AHRQ PSI-90 Footnote', 'AHRQ PSI-90 Score',
             'Total HAC Footnote', 'Total HAC Score', 'Total HAC Score (derived)',
             'Payment Reduction Footnote', 'Payment Reduction', 
             'Payment Reduction (derived)', 'Payment Reduction Reproduced?',
             ]

display_df = display_df.filter(items=items)
display_df = display_df[~display_df['Total HAC Score'].isin([float("NaN"), np.nan])]
display_df = display_df[~display_df['Total HAC Score (derived)'].isin([float("NaN"), np.nan])]
display_df = display_df.round(4)

display_df.sort_values(by=['CAUTI derived score',
                           'CLABSI derived score',
                           'CDI derived score',
                           'MRSA derived score',
                          ], ascending=False, inplace=True)
display_df.head(43)

,Facility ID,file_year,CAUTI Footnote,CAUTI Score,CAUTI derived score,CLABSI Footnote,CLABSI Score,CLABSI derived score,CDI Footnote,CDI Score,CDI derived score,MRSA Footnote,MRSA Score,MRSA derived score,SSI Footnote,SSI Score,AHRQ PSI-90 Footnote,AHRQ PSI-90 Score,Total HAC Footnote,Total HAC Score,Total HAC Score (derived),Payment Reduction Footnote,Payment Reduction,Payment Reduction (derived),Payment Reduction Reproduced?
304,050230,2017,NaN,9.0,9.0,NaN,10.0,10.0,NaN,8.0,7.0,NaN,10.0,10.0,NaN,1.0,NaN,1.0,NaN,6.610,6.440,NaN,Yes,No,0
1678,260025,2017,NaN,9.0,9.0,NaN,10.0,10.0,NaN,4.0,3.0,NaN,10.0,10.0,NaN,1.0,NaN,6.0,NaN,6.680,6.510,NaN,Yes,No,0
3136,500124,2017,NaN,9.0,9.0,NaN,5.0,5.0,NaN,9.0,8.0,NaN,5.0,4.0,NaN,9.0,NaN,2.0,NaN,6.590,6.250,NaN,Yes,No,0
2624,430016,2017,NaN,9.0,9.0,NaN,5.0,5.0,NaN,7.0,6.0,NaN,6.0,5.0,NaN,9.0,NaN,4.0,NaN,6.720,6.380,NaN,Yes,No,0
1454,220163,2017,NaN,9.0,9.0,NaN,4.0,4.0,NaN,8.0,8.0,NaN,5.0,4.0,NaN,4.0,NaN,10.0,NaN,6.600,6.430,NaN,Yes,No,0
1993,330198,2017,NaN,9.0,9.0,NaN,3.0,3.0,NaN,8.0,8.0,NaN,6.0,5.0,NaN,6.0,NaN,8.0,NaN,6.640,6.470,NaN,Yes,No,0
1588,240100,2017,NaN,8.0,8.0,NaN,10.0,10.0,NaN,8.0,7.0,NaN,1.0,1.0,NaN,9.0,NaN,4.0,NaN,6.720,6.550,NaN,Yes,No,0
1104,160033,2017,NaN,8.0,8.0,NaN,10.0,10.0,NaN,4.0,4.0,NaN,7.0,6.0,NaN,9.0,NaN,1.0,NaN,6.610,6.440,NaN,Yes,No,0
719,100256,2017,NaN,8.0,8.0,NaN,9.0,9.0,NaN,10.0,10.0,NaN,6.0,5.0,NaN,5.0,NaN,1.0,NaN,6.610,6.440,NaN,Yes,No,0
2606,420087,2017,NaN,8.0,8.0,NaN,8.0,8.0,NaN,8.0,7.0,NaN,8.0,8.0,NaN,6.0,NaN,1.0,NaN,6.610,6.440,NaN,Yes,No,0


In [16]:
print(sorted(holdout_df['State'].unique()))

['AL', 'AR', 'AZ', 'CA', 'CO', 'FL', 'ID', 'IL', 'IN', 'KY', 'LA', 'MD', 'MN', 'MS', 'NC', 'ND', 'NM', 'NV', 'NY', 'OH', 'OK', 'SC', 'SD', 'TX', 'VA', 'WY']


In [17]:
tdf1 = df_2017[df_2017['Payment Reduction'] == 'Yes']
tdf2 = df_2017[df_2017['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_2017[df_2017['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])

print(df_2017.shape[0])
print(len(df_2017['Facility ID'].unique()))

df_2017.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/P1_HAI_HAC_2017.pkl', protocol=5)
holdout_df.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/P1_HAI_HAC_2017_holdout.pkl', protocol=5) 


0.23985009369144286
0.7601499063085572
3202
3202
